<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:200%;
           font-family:Arial;letter-spacing:0.5px">

<p width = 20%, style="padding: 10px;
              color:white;">
SQL: Subqueries
              
</p>
</div>

Data Science Cohort Live NYC May 2022
<p>Phase 1: Topic 7</p>
<br>
<br>

<div align = "right">
<img src="Images/flatiron-school-logo.png" align = "right" width="200"/>
</div>
    
   

Remember: 

SELECT queries return a new table.

Sometimes:

- Query a table.
- Filter/select/... on that query
- Return result of querying on the first query.

First query is a **subquery**.

Let's get into it with our handy dandy flights database!

In [1]:
import sqlite3
import numpy as np
import pandas as pd

# connect to flights database
con = sqlite3.connect('data/flights.db')

Subqueries can be used to great effect in a variety of circumstances:
    
- Selecting from a table created by a subquery 
    - Subquery in FROM statement
- Filtering on a subquery.
    - Subquery in a WHERE statement
- Filtering in a SELECT statement.


Example of a query containing a subquery:

Get the average of the highest altitude airports in United States, Russia, and China:

- Select airport name, city, and altitude for the highest airport in each country
- Then filter this on the country.
- Then get the average of the altitudes of the resultant airports.

Step by step.

In [2]:
%%capture subquery
pd.read_sql("""

SELECT name, 
     city,
     country,
     MAX(CAST(altitude as float)) AS max_altitude
     
     FROM airports
     
     GROUP BY country
     

""", con)

In [3]:
subquery()

,name,city,country,max_altitude
0,FOB Sharana,Sharan,Afghanistan,7400.0
1,Tirana Rinas,Tirana,Albania,126.0
2,Tamanrasset,Tamanrasset,Algeria,4518.0
3,Fitiuta Airport,Fiti\\'uta,American Samoa,110.0
4,Lubango,Lubango,Angola,5778.0
...,...,...,...,...
235,Jerusalem,Jerusalem,West Bank,2485.0
236,Smara Airport,Smara,Western Sahara,350.0
237,Sanaa Intl,Sanaa,Yemen,7216.0
238,Kasompe,Kasompe,Zambia,4636.0


This is a table produced by the query. We now want to filter this table on country.

- Can select FROM results of above table and filter WHERE conditon is true.
- Subquery creates intermediate table for us.

In [4]:
%%capture query

pd.read_sql("""

SELECT *
    
FROM 

(SELECT name, 
     city,
     country,
     MAX(CAST(altitude as float)) AS max_altitude
     
     FROM airports
     
     GROUP BY country
     
     ) AS high_alt_airp
     
 WHERE high_alt_airp.country IN ("United States", "Russia", "China")



""", con)

In [5]:
query()

,name,city,country,max_altitude
0,Yading Daocheng,Daocheng,China,14472.0
1,Irkutsk-2,Irkutsk,Russia,13411.0
2,Silverton,Silverton,United States,9308.0


Now I select the mean altitude across the three countries.

In [ ]:
%%capture query

pd.read_sql("""

SELECT AVG(high_alt_airp.max_altitude) AS max_altitude_avg
    
FROM 

(SELECT name, 
     city,
     country,
     MAX(CAST(altitude as float)) AS max_altitude
     
     FROM airports
     
     GROUP BY country
     
     ) AS high_alt_airp
     
 WHERE high_alt_airp.country IN ("United States", "Russia", "China")



""", con)

In [ ]:
# Average altitude of the highest airports in Russia, United States, and China
query()

One can create, in principle, a new table of derived columns and do selections and filtering on that new table.

Let's take a look at our routes table again.

In [6]:
pd.read_sql("""

SELECT * 
     FROM routes
     LIMIT 5


""", con)

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2
2,2,2B,410,ASF,2966,MRV,2962,None,0,CR2
3,3,2B,410,CEK,2968,KZN,2990,None,0,CR2
4,4,2B,410,CEK,2968,OVB,4078,None,0,CR2


For each airport I want:
- the count of routes that leaves from there.
- the count of routes that arrive there.

Many ways to do this. But I'll make full use of subqueries and joins here.

In [7]:
departure_table = pd.read_sql("""

SELECT source, COUNT(source) as departure_count
     FROM routes
     
     GROUP BY source
     
""", con)

In [8]:
departure_table

,source,departure_count
0,AAE,9
1,AAL,20
2,AAN,2
3,AAQ,3
4,AAR,8
...,...,...
3404,ZUH,60
3405,ZUM,2
3406,ZVK,3
3407,ZYI,15


In [9]:
arrival_table = pd.read_sql("""

SELECT dest, COUNT(dest) as arrival_count
     FROM routes
     
     GROUP BY dest
     
""", con)

In [10]:
arrival_table

,dest,arrival_count
0,AAE,9
1,AAL,21
2,AAN,2
3,AAQ,3
4,AAR,8
...,...,...
3413,ZUH,59
3414,ZUM,2
3415,ZVK,3
3416,ZYI,15


We can link these the arrival tables and destination tables together:
- Keep records where source and destination are in both tables.
- What operation?

INNER JOIN ON SUBQUERY RESULTS!

In [11]:
%%capture full_result

pd.read_sql("""

SELECT dep.source AS airport_code,
        dep.departure_count,
        arriv.arrival_count 
        
FROM

(SELECT source, COUNT(source) as departure_count
     FROM routes
     
     GROUP BY source) AS dep

INNER JOIN 

(SELECT dest, COUNT(dest) as arrival_count
     FROM routes
     
     GROUP BY dest) AS arriv

ON dep.source = arriv.dest
     
""", con)

In [12]:
full_result()

,airport_code,departure_count,arrival_count
0,AAE,9,9
1,AAL,20,21
2,AAN,2,2
3,AAQ,3,3
4,AAR,8,8
...,...,...,...
3397,ZUH,60,59
3398,ZUM,2,2
3399,ZVK,3,3
3400,ZYI,15,15


This result is a derived table telling us useful information for each airport and keeping relevant records.

- We can filter or aggregate on this derived table as we see fit.


- Get all airports with high amount of routes servicing it: 
    - departure and arrival count both greater than 100.
- Order by arrival and then deperature count both in descending order.

Help this hapless data science instructor out!

In [23]:
%%capture filtered_result

pd.read_sql("""

SELECT dep.source AS airport_code,
        dep.departure_count,
        arriv.arrival_count 
        
FROM

(SELECT source, COUNT(source) as departure_count
     FROM routes
     
     GROUP BY source) AS dep

INNER JOIN 

(SELECT dest, COUNT(dest) as arrival_count
     FROM routes
     
     GROUP BY dest) AS arriv

ON dep.source = arriv.dest

WHERE dep.departure_count > 100 AND arriv.arrival_count > 100

ORDER BY arriv.arrival_count DESC, dep.departure_count DESC
     
""", con)

In [24]:
filtered_result()

,airport_code,departure_count,arrival_count
0,ATL,915,911
1,ORD,558,550
2,PEK,535,534
3,LHR,527,524
4,CDG,524,517
...,...,...,...
156,MAA,107,104
157,MCT,104,104
158,SJU,102,102
159,CMB,101,102


- Complex as subqueries pile on. 
- There are ways to organize subqueries: common table expressions
- Will make code readable.

But first: other ways subqueries can be used.

#### Subqueries in the WHERE statement

- Can use the result of a subquery to filter another selection.

I want all route records where the departure airport has a departure route count greater than 200.

In [36]:
%%capture where_subquery1
pd.read_sql('''


SELECT *
            
FROM routes

WHERE source IN (SELECT source 
     FROM routes
     GROUP BY source
     HAVING COUNT(source) > 200)
''', con)

In [37]:
where_subquery1()

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,5,2B,410,DME,4029,KZN,2990,None,0,CR2
1,6,2B,410,DME,4029,NBC,6969,None,0,CR2
2,7,2B,410,DME,4029,TGK,\N,None,0,CR2
3,8,2B,410,DME,4029,UUA,6160,None,0,CR2
4,123,2L,2750,ZRH,1678,BDS,1506,None,0,100
...,...,...,...,...,...,...,...,...,...,...
21698,67646,ZL,4178,SYD,3361,OAG,6793,None,0,SF3
21699,67647,ZL,4178,SYD,3361,PKE,6317,None,0,SF3
21700,67648,ZL,4178,SYD,3361,TRO,6794,None,0,SF3
21701,67649,ZL,4178,SYD,3361,WGA,3363,None,0,SF3


Subquerying made this query possible.

#### The Semijoin

- Using a subquery from one table to filter another table.

Example:

Get the airport name, code, country, and altitude for airports with more than 200 routes originating from that airport.

In [38]:
%%capture semijoin
pd.read_sql('''

SELECT *
FROM airports

WHERE code IN (
SELECT source 
     FROM routes
     GROUP BY source
     HAVING COUNT(source) > 200 
)
''', con)

In [39]:
semijoin()

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,191,193,Lester B Pearson Intl,Toronto,Canada,YYZ,CYYZ,43.677223,-79.630556,569,-5,A,America/Toronto
1,299,302,Brussels Natl,Brussels,Belgium,BRU,EBBR,50.901389,4.484444,184,1,E,Europe/Brussels
2,337,340,Frankfurt Main,Frankfurt,Germany,FRA,EDDF,50.026421,8.543125,364,1,E,Europe/Berlin
3,342,345,Dusseldorf,Duesseldorf,Germany,DUS,EDDL,51.289453,6.766775,147,1,E,Europe/Berlin
4,343,346,Franz Josef Strauss,Munich,Germany,MUC,EDDM,48.353783,11.786086,1487,1,E,Europe/Berlin
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,3778,3877,Mc Carran Intl,Las Vegas,United States,LAS,KLAS,36.080056,-115.15225,2141,-8,A,America/Los_Angeles
64,3779,3878,Orlando Intl,Orlando,United States,MCO,KMCO,28.429394,-81.308994,96,-5,A,America/New_York
65,3786,3885,Suvarnabhumi Intl,Bangkok,Thailand,BKK,VTBS,13.681108,100.747283,5,7,U,Asia/Bangkok
66,3890,3998,Son Sant Joan,Palma de Mallorca,Spain,PMI,LEPA,39.55361,2.727778,24,1,E,Europe/Madrid


#### The antijoin

Just the negation of the semijoin:

In [40]:
%%capture antijoin

pd.read_sql('''
SELECT *
FROM airports

WHERE NOT code IN (
SELECT source 
     FROM routes
     GROUP BY source
     HAVING COUNT(source) > 300 
)
''', con)

In [41]:
antijoin()

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5843,8102,9537,Mansons Landing Water Aerodrome,Mansons Landing,Canada,YMU,\N,50.066667,-124.983333,0,-8,A,America/Vancouver
5844,8103,9538,Port McNeill Airport,Port McNeill,Canada,YMP,\N,50.575556,-127.028611,225,-8,A,America/Vancouver
5845,8104,9539,Sullivan Bay Water Aerodrome,Sullivan Bay,Canada,YTG,\N,50.883333,-126.833333,0,-8,A,America/Vancouver
5846,8105,9540,Deer Harbor Seaplane,Deer Harbor,United States,DHB,\N,48.618397,-123.00596,0,-8,A,America/Los_Angeles


#### Subqueries in the SELECT statement

Get altitude of airport side by side with average airport altitude in the respective country.

In [42]:
%%capture select_subquery

pd.read_sql('''
SELECT country,
name, 
CAST(altitude as int) AS altitude ,

(SELECT AVG(CAST(altitude as int))
FROM airports
GROUP BY 
country) AS avg_alt

FROM airports


''', con)

In [43]:
select_subquery()

,country,name,altitude,avg_alt
0,Papua New Guinea,Goroka,5282,3750.809524
1,Papua New Guinea,Madang,20,3750.809524
2,Papua New Guinea,Mount Hagen,5388,3750.809524
3,Papua New Guinea,Nadzab,239,3750.809524
4,Papua New Guinea,Port Moresby Jacksons Intl,146,3750.809524
...,...,...,...,...
8102,Canada,Mansons Landing Water Aerodrome,0,3750.809524
8103,Canada,Port McNeill Airport,225,3750.809524
8104,Canada,Sullivan Bay Water Aerodrome,0,3750.809524
8105,United States,Deer Harbor Seaplane,0,3750.809524


#### Common Table Expressions (CTEs)

Useful for when:

- Multiple subqueries floating around.
- Reusing same subqueries over and over again in same statement.

Cleans code up, reduces error, makes readable.

CTE syntax:

WITH subquery1_name AS (subquery1 statement),
<br>
WITH subquery2_name AS (subquery2 statement), 
<br>
...., 
<br>
WITH subquery_n_name AS (subquery_n statement)
<br>
<br>
MAIN QUERY

In [ ]:
%%capture filtered_result

pd.read_sql("""

WITH s1 AS (
     SELECT source, COUNT(source) as departure_count
     FROM routes
     GROUP BY source),
     
     s2 AS ( 
     SELECT dest, COUNT(dest) as arrival_count
     FROM routes
     GROUP BY dest
     )


SELECT source, departure_count, arrival_count
FROM
s1 INNER JOIN s2
ON s1.source = s2.dest

     
""", con)

In [ ]:
filtered_result()

CTEs make the primary query easy to read.

- Subqueries can just be referenced by looking it up in the CTE.